In [ ]:
!pip install tensorflow-gpu -U
# !pip install tensorflow-gpu==2.0a0

In [ ]:
import tensorflow as tf
print(tf.__version__)
#is it using the gpu?
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

In [ ]:
# !pip install keras -U

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install megnet 

In [ ]:
!pip show keras

In [ ]:
conda install -c openbabel openbabel

In [ ]:
# import sys
# !conda install --yes --prefix {sys.prefix} -c openbabel openbabel==2.4.1


In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} -c rdkit rdkit==2018.03.3

In [ ]:
sys.prefix

In [ ]:
# # only reading 10% of data for debug
# train = pd.read_csv('../input/train.csv')
# test = pd.read_csv('../input/champs-scalar-coupling/test.csv')
# len(train['molecule_name'].unique())

In [ ]:
# test['id'].count()

In [ ]:
# structure = pd.read_csv('../input/structures.csv')
# len(structure['molecule_name'].unique())

In [ ]:
# from keras_radam import RAdam

In [ ]:
# train = pd.read_csv('../input/champs-scalar-coupling/train.csv')
max_value =204.88 
min_value = -36.2186



print(max_value,min_value)

graph code

In [ ]:
"""Abstract classes and utility operations for building graph representations and
data loaders (known as Sequence objects in Keras).

Most users will not need to interact with this module."""
import json


from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import Dense, Input, Concatenate, Add, Embedding, Dropout
from megnet.layers import MEGNetLayer, Set2Set
from megnet.activations import softplus2
from keras.regularizers import l2
from keras.backend import int_shape
from megnet.callbacks import ModelCheckpointMAE, ManualStop, ReduceLRUponNan
from megnet.data.graph import GraphBatchDistanceConvert, GraphBatchGenerator, GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.utils.preprocessing import DummyScaler
import numpy as np
import os
from warnings import warn
from monty.serialization import dumpfn, loadfn
from abc import ABCMeta, abstractmethod
from operator import itemgetter
from megnet.utils.general import expand_1st, to_list
from monty.json import MSONable
from megnet.data import local_env
from inspect import signature
from pymatgen.analysis.local_env import NearNeighbors
from keras.utils import Sequence

from keras.models import Model
import pickle
from sklearn.model_selection import train_test_split

import pandas as pd

import six
# #from pybel import *
import openbabel as ob
import pybel
from multiprocessing import Pool
from tqdm import *
from pathlib import Path
import sys
import itertools
from typing import List
from functools import partial
from collections import deque
from pymatgen import Molecule, Element
from pymatgen.io.babel import BabelMolAdaptor
from megnet.data.qm9 import ring_to_vector
from megnet.utils.general import fast_label_binarize


class StructureGraph(MSONable):
    """
    This is a base class for converting converting structure into graphs or model inputs
    Methods to be implemented are follows:
        1. convert(self, structure)
            This is to convert a structure into a graph dictionary
        2. get_input(self, structure)
            This method convert a structure directly to a model input
        3. get_flat_data(self, graphs, targets)
            This method process graphs and targets pairs and output model input list.

    """

    # TODO (wardlt): Consider making "num_*_features" funcs to simplify making a MEGNet model

    def __init__(self,
                 nn_strategy=None,
                 atom_converter=None,
                 bond_converter=None,
                 **kwargs):

        if isinstance(nn_strategy, str):
            strategy = local_env.get(nn_strategy)
            parameters = signature(strategy).parameters
            param_dict = {i: j.default for i, j in parameters.items()}
            for i, j in kwargs.items():
                if i in param_dict:
                    setattr(self, i, j)
                    param_dict.update({i: j})
            self.nn_strategy = strategy(**param_dict)
        elif isinstance(nn_strategy, NearNeighbors):
            self.nn_strategy = nn_strategy
        elif nn_strategy is None:
            self.nn_strategy = None
        else:
            raise RuntimeError("Strategy not valid")

        self.atom_converter = atom_converter
        self.bond_converter = bond_converter
        if self.atom_converter is None:
            self.atom_converter = self._get_dummy_converter()
        if self.bond_converter is None:
            self.bond_converter = self._get_dummy_converter()

    def convert(self, structure, state_attributes=None):
        """
        Take a pymatgen structure and convert it to a index-type graph representation
        The graph will have node, distance, index1, index2, where node is a vector of Z number
        of atoms in the structure, index1 and index2 mark the atom indices forming the bond and separated by
        distance.
        For state attributes, you can set structure.state = [[xx, xx]] beforehand or the algorithm would
        take default [[0, 0]]

        Args:
            state_attributes: (list) state attributes
            structure: (pymatgen structure)
            (dictionary)
        """
        state_attributes = state_attributes or [[0, 0]]
        index1 = []
        index2 = []
        bonds = []
        if self.nn_strategy is None:
            raise RuntimeError("NearNeighbor strategy is not provided!")
        for n, neighbors in enumerate(self.nn_strategy.get_all_nn_info(structure)):
            index1.extend([n] * len(neighbors))
            for neighbor in neighbors:
                index2.append(neighbor['site_index'])
                bonds.append(neighbor['weight'])

        atoms = [i.specie.Z for i in structure]

        if np.size(np.unique(index1)) < len(atoms):
            raise RuntimeError("Isolated atoms found in the structure")
        else:
            return {'atom': np.array(atoms, dtype='int32').tolist(),
                    'bond': bonds,
                    'state': state_attributes,
                    'index1': index1,
                    'index2': index2
                    }

    def convert(self, structure, attributes, state_attributes=None):
        """
        Take a pymatgen structure and convert it to a index-type graph representation
        The graph will have node, distance, index1, index2, where node is a vector of Z number
        of atoms in the structure, index1 and index2 mark the atom indices forming the bond and separated by
        distance.
        For state attributes, you can set structure.state = [[xx, xx]] beforehand or the algorithm would
        take default [[0, 0]]

        Args:
            state_attributes: (list) state attributes
            structure: (pymatgen structure)
            (dictionary)
        """
        state_attributes = state_attributes or [[0, 0]]
        index1 = []
        index2 = []
        bonds = []
        if self.nn_strategy is None:
            raise RuntimeError("NearNeighbor strategy is not provided!")
        for n, neighbors in enumerate(self.nn_strategy.get_all_nn_info(structure)):
            index1.extend([n] * len(neighbors))
            for neighbor in neighbors:
                index2.append(neighbor['site_index'])
                bonds.append(neighbor['weight'])

        atoms = [i.specie.Z for i in structure]

        if np.size(np.unique(index1)) < len(atoms):
            raise RuntimeError("Isolated atoms found in the structure")
        else:
            return {'atom': np.array(atoms, dtype='int32').tolist(),
                    'bond': bonds,
                    'state': state_attributes,
                    'index1': index1,
                    'index2': index2
                    }

    def __call__(self, structure):
        return self.convert(structure)

    def get_input(self, structure):
        """
        Turns a structure into model input
        """
        graph = self.convert(structure)
        return self.graph_to_input(graph)

    def graph_to_input(self, graph):
        """
        Turns a graph into model input

        Args:
            (dict): Dictionary description of the graph
        Return:
            ([np.ndarray]): Inputs in the form needed by MEGNet
        """
        gnode = [0] * len(graph['atom'])
        gbond = [0] * len(graph['index1'])

        return [expand_1st(self.atom_converter.convert(graph['atom'])),
                expand_1st(self.bond_converter.convert(graph['bond'])),
                expand_1st(np.array(graph['state'])),
                expand_1st(np.array(graph['index1'])),
                expand_1st(np.array(graph['index2'])),
                expand_1st(np.array(gnode)),
                expand_1st(np.array(gbond))]

    def get_flat_data(self, graphs, targets=None):
        """
        Expand the graph dictionary to form a list of features and targets tensors.
        This is useful when the model is trained on assembled graphs on the fly.

        Args:
            graphs: (list of dictionary) list of graph dictionary for each structure
            targets: (list of float or list) Optional: corresponding target
                values for each structure

        Returns:
            tuple(node_features, edges_features, global_values, index1, index2, targets)
        """

        output = []  # Will be a list of arrays

        # Convert the graphs to matrices
        for feature in ['atom', 'bond', 'state', 'index1', 'index2']:
            output.append([np.array(x[feature]) for x in graphs])

        # If needed, add the targets
        if targets is not None:
            output.append([to_list(t) for t in targets])

        return tuple(output)

    def get_flat_scarlar_data(self, graphs, targets=None):
        """
        Expand the graph dictionary to form a list of features and targets tensors.
        This is useful when the model is trained on assembled graphs on the fly.

        Args:
            graphs: (list of dictionary) list of graph dictionary for each structure
            targets: (list of float or list) Optional: corresponding target
                values for each structure

        Returns:
            tuple(node_features, edges_features, global_values, index1, index2, targets)
        """

        output = []  # Will be a list of arrays

        # Convert the graphs to matrices
        for feature in ['atom', 'bond', 'state', 'index1', 'index2']:
            output.append([np.array(x[feature]) for x in graphs])

        #  gnode = [0] * len(graph['atom'])
        # gbond = [0] * len(graph['index1'])
        # elf.graph_converter.graph_to_input(graph)
        # # If needed, add the targets
        # count=128
        # if targets is not None:
        #     for bond in output[1]:
        #         c = len(bond)
        #         if(c>count):
        #             count = c
        # print("bond max count:"+str(c))
        # current_words = list(current_words + [0] * (10 - len(current_words)))
        # output.append([list(to_list(t['scalar_coupling_constant'])+[0]*(count-len(to_list(t['scalar_coupling_constant'])))) for t in targets])

        for feature in ['output']:
            ff = np.array([np.array(x[feature]) for x in targets])
            print('mmmmm')
            print(ff.shape)

            # scal_coupling_constant = ff.reshape(-1, 1)
            # print(scal_coupling_constant.shape)

            output.append(ff)
            print(ff)

        return tuple(output)

    @staticmethod
    def _get_dummy_converter():
        return DummyConverter()

    def as_dict(self):
        all_dict = super().as_dict()
        if 'nn_strategy' in all_dict:
            nn_strategy = all_dict.pop('nn_strategy')
            all_dict.update({'nn_strategy': local_env.serialize(nn_strategy)})
        return all_dict

    @classmethod
    def from_dict(cls, d):
        if 'nn_strategy' in d:
            nn_strategy = d.pop('nn_strategy')
            nn_strategy_obj = local_env.deserialize(nn_strategy)
            d.update({'nn_strategy': nn_strategy_obj})
            return super().from_dict(d)
        return super().from_dict(d)


class DistanceConverter(MSONable):
    """
    Base class for distance conversion. The class needs to have a convert method.
    """

    def convert(self, d):
        raise NotImplementedError


class DummyConverter(DistanceConverter):
    """
    Dummy converter as a placeholder
    """

    def convert(self, d):
        return d


class GaussianDistance(DistanceConverter):
    """
    Expand distance with Gaussian basis sit at centers and with width 0.5.

    Args:
        centers: (np.array)
        width: (float)
    """

    def __init__(self, centers=np.linspace(0, 5, 100), width=0.5):
        self.centers = centers
        self.width = width

    def convert(self, d):
        """
        expand distance vector d with given parameters

        Args:
            d: (1d array) distance array

        Returns
            (matrix) N*M matrix with N the length of d and M the length of centers
        """
        d = np.array(d)
        return np.exp(-(d[:, None] - self.centers[None, :]) ** 2 / self.width ** 2)


class MoorseLongRange(DistanceConverter):
    """
    This is an attempt to implement a Moorse/long range interactomic potential like
    distance expansion. The distance is expanded with this basis at different equilibrium
    bond distance, r_eq. It is still a work in progress. Do not use if you do not know
    much about the parameters
    ref: https://en.wikipedia.org/wiki/Morse/Long-range_potential#Function

    Args:
        d_e: (float) dissociate energy
        r_ref: (float) reference bond length
        r_eq: (list) equilibrium bond length
        p: (int) exponential term in the original equation, see ref
        q: (int) exponential term in the original equaiton, see ref
        cm: (list) long range coefficients in u_LR = \Sigma_i_N (cm_i / r^i)
        betas: (list) parameters determining the transition between long range and short range
    """

    def __init__(self, d_e=1, r_ref=2, r_eq=[1, 2, 3],
                 p=2, q=2, cm=[1, 2, 3, 4],
                 betas=[0.1, 0.2, 0.3, 0.4]):
        self.d_e = d_e
        self.r_ref = r_ref
        self.r_eq = np.array(r_eq)
        self.p = p
        self.q = q
        self.cm = np.array(cm).ravel()
        self.n_cm = len(self.cm)
        self.betas = np.array(betas).ravel()

    def convert(self, d):
        return self.d_e * (1 - self.u(d)[:, None] / self.u(self.r_eq)[None, :] *
                           np.exp(-self.beta(d) * self.y(d[:, None], self.r_eq[None, :], self.p))) ** 2

    def u(self, r):
        m_i = np.arange(1, self.n_cm + 1)
        if np.size(r) == 1:
            return np.sum(self.cm / r ** m_i)
        return np.sum(self.cm[None, :] / r[:, None] ** m_i[None, :], axis=1).ravel()

    @staticmethod
    def y(r, r_ref, p):
        return (r ** p - r_ref ** p) / (r ** p + r_ref ** p)

    def beta(self, r):
        y_p_ref = self.y(r, self.r_ref, self.p)
        y_q_ref = self.y(r, self.r_ref, self.q)
        return self.beta_inf[None, :] * y_p_ref[:, None] + (1 - y_p_ref[:, None]) * \
               np.sum(self.betas[None, :] * y_q_ref[:, None] ** np.arange(0, len(self.betas))[None, :], axis=1).ravel()[
               :, None]

    @property
    def beta_inf(self):
        return np.log(2 * self.d_e / self.u(self.r_eq))


class BaseGraphBatchGenerator(Sequence):
    """Base class for classes that generate batches of training data for MEGNet.
    Based on the Sequence class, which is the data loader equivalent for Keras.

    Implementations of this base class must implement the :meth:`_generate_inputs`,
    which generates the lists of graph descriptions for a batch.

    The :meth:`process_atom_features` function and related functions are used to modify
    the features for each atom, bond, and global features when creating a batch.
    """

    def __init__(self, dataset_size, targets, batch_size=128, shuffle=True):
        """
        Args:
            dataset_size (int): Number of entries in dataset
            targets (ndarray): Feature to be predicted for each network
            batch_size (int): Maximum batch size
            shuffle (bool): Whether to shuffle the data after each step
        """
        if targets is not None:
            self.targets = np.array(targets)
        else:
            self.targets = None
        self.batch_size = batch_size
        self.total_n = dataset_size
        self.is_shuffle = shuffle
        self.max_step = int(np.ceil(self.total_n / batch_size))
        self.mol_index = np.arange(self.total_n)
        if self.is_shuffle:
            self.mol_index = np.random.permutation(self.mol_index)

    def __len__(self):
        return self.max_step

    def _combine_graph_data(self, feature_list_temp, connection_list_temp, global_list_temp,
                            index1_temp, index2_temp):
        """Compile the matrices describing each graph into single matrices for the entire graph

        Beyond concatenating the graph descriptions, this operation updates the indices of each
        node to be sequential across all graphs so they are not duplicated between graphs

        Args:
            feature_list_temp ([ndarray]): List of features for each node
            connection_list_temp ([ndarray]): List of features for each connection
            global_list_temp ([ndarray]): List of global state for each graph
            index1_temp ([ndarray]): List of indices for the start of each bond
            index2_temp ([ndarray]): List of indices for the end of each bond
        Returns:
            (tuple): Input arrays describing the entire batch of networks:
                - ndarray: Features for each node
                - ndarray: Features for each connection
                - ndarray: Global state for each graph
                - ndarray: Indices for the start of each bond
                - ndarray: Indices for the end of each bond
                - ndarray: Index of graph associated with each node
                - ndarray: Index of graph associated with each connection
        """
        # get atom's structure id
        gnode = []
        for i, j in enumerate(feature_list_temp):
            gnode += [i] * len(j)
        # get bond features from a batch of structures
        # get bond's structure id
        gbond = []
        for i, j in enumerate(connection_list_temp):
            gbond += [i] * len(j)

        # assemble atom features together
        feature_list_temp = np.concatenate(feature_list_temp, axis=0)
        feature_list_temp = self.process_atom_feature(feature_list_temp)

        # assemble bond feature together
        connection_list_temp = np.concatenate(connection_list_temp, axis=0)
        connection_list_temp = self.process_bond_feature(connection_list_temp)

        # assemble state feature together
        global_list_temp = np.concatenate(global_list_temp, axis=0)
        global_list_temp = self.process_state_feature(global_list_temp)

        # assemble bond indices
        index1 = []
        index2 = []
        offset_ind = 0
        for ind1, ind2 in zip(index1_temp, index2_temp):
            index1 += [i + offset_ind for i in ind1]
            index2 += [i + offset_ind for i in ind2]
            offset_ind += (max(ind1) + 1)

        # Compile the inputs in needed order
        inputs = [expand_1st(feature_list_temp),
                  expand_1st(connection_list_temp),
                  expand_1st(global_list_temp),
                  expand_1st(index1),
                  expand_1st(index2),
                  expand_1st(gnode),
                  expand_1st(gbond)]
        return inputs

    def on_epoch_end(self):
        if self.is_shuffle:
            self.mol_index = np.random.permutation(self.mol_index)

    def process_atom_feature(self, x):
        return x

    def process_bond_feature(self, x):
        return x

    def process_state_feature(self, x):
        return x

    def __getitem__(self, index):
        # Get the indices for this batch
        batch_index = self.mol_index[index * self.batch_size:(index + 1) * self.batch_size]

        # Get the inputs for each batch
        inputs = self._generate_inputs(batch_index)

        # Make the graph data
        inputs = self._combine_graph_data(*inputs)

        # Return the batch
        if self.targets is None:
            return inputs
        else:
            # get targets
            it = itemgetter(*batch_index)
            target_temp = itemgetter_list(self.targets, batch_index)
            target_temp = np.atleast_2d(target_temp)

            return inputs, expand_1st(target_temp)

    @abstractmethod
    def _generate_inputs(self, batch_index):
        """Get the graph descriptions for each batch

        Args:
             batch_index ([int]): List of indices for training batch
        Returns:
            (tuple): Input arrays describing each network:
                - [ndarray]: List of features for each node
                - [ndarray]: List of features for each connection
                - [ndarray]: List of global state for each graph
                - [ndarray]: List of indices for the start of each bond
                - [ndarray]: List of indices for the end of each bond
        """
        pass


class GraphBatchGenerator(BaseGraphBatchGenerator):
    """
    A generator class that assembles several structures (indicated by
    batch_size) and form (x, y) pairs for model training.

    Args:
        atom_features: (list of np.array) list of atom feature matrix,
        bond_features: (list of np.array) list of bond features matrix
        state_features: (list of np.array) list of [1, G] state features,
            where G is the global state feature dimension
        index1_list: (list of integer) list of (M, ) one side atomic index of the bond,
        M is different for different structures
        index2_list: (list of integer) list of (M, ) the other side atomic
            index of the bond, M is different for different structures,
            but it has to be the same as the corresponding index1.
        targets: (numpy array), N*1, where N is the number of structures
        batch_size: (int) number of samples in a batch
    """

    def __init__(self,
                 atom_features,
                 bond_features,
                 state_features,
                 index1_list,
                 index2_list,
                 targets=None,
                 batch_size=128,
                 is_shuffle=True):
        super().__init__(len(atom_features), targets, batch_size, is_shuffle)
        self.atom_features = atom_features
        self.bond_features = bond_features
        self.state_features = state_features
        self.index1_list = index1_list
        self.index2_list = index2_list

    def _generate_inputs(self, batch_index):
        """Get the graph descriptions for each batch

        Args:
             batch_index ([int]): List of indices for training batch
        Returns:
            (tuple): Input arrays describe each network:
                - [ndarray]: List of features for each nodes
                - [ndarray]: List of features for each connection
                - [ndarray]: List of global state for each graph
                - [ndarray]: List of indices for the start of each bond
                - [ndarray]: List of indices for the end of each bond
        """

        # Get the features and connectivity lists for this batch
        it = itemgetter(*batch_index)
        feature_list_temp = itemgetter_list(self.atom_features, batch_index)
        connection_list_temp = itemgetter_list(self.bond_features, batch_index)
        global_list_temp = itemgetter_list(self.state_features, batch_index)
        index1_temp = itemgetter_list(self.index1_list, batch_index)
        index2_temp = itemgetter_list(self.index2_list, batch_index)

        return feature_list_temp, connection_list_temp, global_list_temp, \
               index1_temp, index2_temp


class GraphBatchDistanceConvert(GraphBatchGenerator):
    """
    Generate batch of structures with bond distance being expanded using a Expansor

    Args:
        atom_features: (list of np.array) list of atom feature matrix,
        bond_features: (list of np.array) list of bond features matrix
        state_features: (list of np.array) list of [1, G] state features, where G is the global state feature dimension
        index1_list: (list of integer) list of (M, ) one side atomic index of the bond, M is different for differentstructures
        index2_list: (list of integer) list of (M, ) the other side atomic index of the bond, M is different for different
            structures, but it has to be the same as the correponding index1.
        targets: (numpy array), N*1, where N is the number of structures
        batch_size: (int) number of samples in a batch
        is_shuffle: (bool) whether to shuffle the structure, default to True
        distance_converter: (bool) converter for processing the distances

    """

    def __init__(self,
                 atom_features,
                 bond_features,
                 state_features,
                 index1_list,
                 index2_list,
                 targets=None,
                 batch_size=128,
                 is_shuffle=True,
                 distance_converter=None):
        super().__init__(atom_features=atom_features,
                         bond_features=bond_features,
                         state_features=state_features,
                         index1_list=index1_list,
                         index2_list=index2_list,
                         targets=targets,
                         batch_size=batch_size,
                         is_shuffle=is_shuffle)
        self.distance_converter = distance_converter

    def process_bond_feature(self, x):
        return self.distance_converter.convert(x)


def itemgetter_list(l, indices):
    """
    Get indices of l and return a tuple

    Args:
        l:  (list)
        indices: (list) indices

    Returns:
        (tuple)
    """
    it = itemgetter(*indices)
    if np.size(indices) == 1:
        return it(l),
    else:
        return it(l)


"""
Tools for creating graph inputs from molecule data
"""

# from megnet.data.graph import (StructureGraph, GaussianDistance,
#                                BaseGraphBatchGenerator, GraphBatchGenerator)
#
try:
    import pybel
except ImportError:
    pybel = None

try:
    from rdkit import Chem
except ImportError:
    Chem = None

__date__ = '12/01/2018'

# List of features to use by default for each atom
_ATOM_FEATURES = ['element', 'chirality', 'formal_charge', 'ring_sizes',
                  'hybridization', 'donor', 'acceptor', 'aromatic']

# List of features to use by default for each bond
_BOND_FEATURES = ['bond_type', 'same_ring', 'spatial_distance', 'graph_distance', 'coupling_type']
# _BOND_FEATURES = ['bond_type', 'same_ring', 'spatial_distance', 'graph_distance']

# List of elements in library to use by default
_ELEMENTS = ['H', 'C', 'N', 'O', 'F']


class SimpleMolGraph(StructureGraph):
    """
    Default using all atom pairs as bonds. The distance between atoms are used
    as bond features. By default the distance is expanded using a Gaussian
    expansion with centers at np.linspace(0, 4, 20) and width of 0.5
    """

    def __init__(self,
                 nn_strategy='AllAtomPairs',
                 atom_converter=None,
                 bond_converter=None
                 ):
        if bond_converter is None:
            bond_converter = GaussianDistance(np.linspace(0, 4, 20), 0.5)
        super().__init__(nn_strategy=nn_strategy, atom_converter=atom_converter,
                         bond_converter=bond_converter)


class MolecularGraph(StructureGraph):
    """Class for generating the graph inputs from a molecule

    Computes many different features for the atoms and bonds in a molecule, and prepares them
    in a form compatible with MEGNet models. The :meth:`convert` method takes a OpenBabel molecule
    and, besides computing features, also encodes them in a form compatible with machine learning.
    Namely, the `convert` method one-hot encodes categorical variables and concatenates
    the atomic features

    ## Atomic Features

    This class can compute the following features for each atom

    - `atomic_num`: The atomic number
    - `element`: (categorical) Element identity. (Unlike `atomic_num`, element is one-hot-encoded)
    - `chirality`: (categorical) R, S, or not a Chiral center (one-hot encoded).
    - `formal_charge`: Formal charge of the atom
    - `ring_sizes`: For rings with 9 or fewer atoms, how many unique rings
    of each size include this atom
    - `hybridization`: (categorical) Hybridization of atom: sp, sp2, sp3, sq.
    planer, trig, octahedral, or hydrogen
    - `donor`: (boolean) Whether the atom is a hydrogen bond donor
    - `acceptor`: (boolean) Whether the atom is a hydrogen bond acceptor
    - `aromatic`: (boolean) Whether the atom is part of an aromatic system

    ## Atom Pair Features

    The class also computes features for each pair of atoms

    - `bond_type`: (categorical) Whether the pair are unbonded, or in a single, double, triple, or aromatic bond
    - `same_ring`: (boolean) Whether the atoms are in the same aromatic ring
    - `graph_distance`: Distance of shortest path between atoms on the bonding graph
    - `spatial_distance`: Euclidean distance between the atoms. By default, this distance is expanded into
        a vector of 20 different values computed using the `GaussianDistance` converter

    """

    def __init__(self, atom_features=None, bond_features=None, distance_converter=None,
                 known_elements=None):
        """
        Args:
            atom_features ([str]): List of atom features to compute
            bond_features ([str]): List of bond features to compute
            distance_converter (DistanceCovertor): Tool used to expand distances
                from a single scalar vector to an array of values
            known_elements ([str]): List of elements expected to be in dataset. Used only if the
                feature `element` is used to describe each atom
        """

        # Check if openbabel and RDKit are installed
        if Chem is None or pybel is None:
            raise RuntimeError('RDKit and openbabel must be installed')

        super().__init__()
        if bond_features is None:
            bond_features = _BOND_FEATURES
        if atom_features is None:
            atom_features = _ATOM_FEATURES
        if distance_converter is None:
            distance_converter = GaussianDistance(np.linspace(0, 4, 20), 0.5)
        if known_elements is None:
            known_elements = _ELEMENTS

        # Check if all feature names are valid
        if any(i not in _ATOM_FEATURES for i in atom_features):
            bad_features = set(atom_features).difference(_ATOM_FEATURES)
            raise ValueError('Unrecognized atom features: {}'.format(', '.join(bad_features)))
        self.atom_features = atom_features
        if any(i not in _BOND_FEATURES for i in bond_features):
            bad_features = set(bond_features).difference(_BOND_FEATURES)
            raise ValueError('Unrecognized bond features: {}'.format(', '.join(bad_features)))
        self.bond_features = bond_features
        self.known_elements = known_elements
        self.distance_converter = distance_converter

    def convert(self, mol, state_attributes=None, full_pair_matrix=True):
        """
        Compute the representation for a molecule

        Args：
            mol (pybel.Molecule): Molecule to generate features for
            state_attributes (list): State attributes. Uses average mass and number of bonds per atom as default
            full_pair_matrix (bool): Whether to generate info for all atom pairs, not just bonded ones
        Returns:
            (dict): Dictionary of features
        """

        # Get the features features for all atoms and bonds
        atom_features = []
        atom_pairs = []
        for idx, atom in enumerate(mol.atoms):
            f = self.get_atom_feature(mol, atom)
            atom_features.append(f)
        atom_features = sorted(atom_features, key=lambda x: x["coordid"])
        num_atoms = mol.OBMol.NumAtoms()
        for i, j in itertools.combinations(range(0, num_atoms), 2):
            bond_feature = self.get_pair_feature(mol, i, j, full_pair_matrix)
            if bond_feature:
                atom_pairs.append(bond_feature)
            else:
                continue

        # Compute the graph distance, if desired
        if 'graph_distance' in self.bond_features:
            graph_dist = self._dijkstra_distance(atom_pairs)
            for i in atom_pairs:
                i.update({'graph_distance': graph_dist[i['a_idx'], i['b_idx']]})

        # Generate the state attributes (that describe the whole network)
        state_attributes = state_attributes or [
            [mol.molwt / num_atoms,
             len([i for i in atom_pairs if i['bond_type'] > 0]) / num_atoms]
        ]

        # Get the atom features in the order they are requested by the user as a 2D array
        atoms = []
        for atom in atom_features:
            atoms.append(self._create_atom_feature_vector(atom))

        # Get the bond features in the order request by the user
        bonds = []
        index1_temp = []
        index2_temp = []
        for bond in atom_pairs:
            # Store the index of each bond
            index1_temp.append(bond.pop('a_idx'))
            index2_temp.append(bond.pop('b_idx'))

            # Get the desired bond features
            bonds.append(self._create_pair_feature_vector(bond))

        # Given the bonds (i,j), make it so (i,j) == (j, i)
        index1 = index1_temp + index2_temp
        index2 = index2_temp + index1_temp
        bonds = bonds + bonds

        # Sort the arrays by the beginning index
        sorted_arg = np.argsort(index1)
        index1 = np.array(index1)[sorted_arg].tolist()
        index2 = np.array(index2)[sorted_arg].tolist()
        bonds = np.array(bonds)[sorted_arg].tolist()

        return {'atom': atoms,
                'bond': bonds,
                'state': state_attributes,
                'index1': index1,
                'index2': index2}

    def convert_coupling1(self, mol, coupling_attributes, energy_dipole, charges_magnetic, target_convert=True,
                          state_attributes=None, full_pair_matrix=True):
        """
        Compute the representation for a molecule

        Args：
            mol (pybel.Molecule): Molecule to generate features for
            state_attributes (list): State attributes. Uses average mass and number of bonds per atom as default
            full_pair_matrix (bool): Whether to generate info for all atom pairs, not just bonded ones
        Returns:
            (dict): Dictionary of features
        """

        # Get the features features for all atoms and bonds
        atom_features = []
        atom_pairs = []
        for idx, atom in enumerate(mol.atoms):
            get_data = charges_magnetic[charges_magnetic['atom_index'] == idx]
            f = self.get_atom_feature(mol, get_data, atom)
            atom_features.append(f)
        atom_features = sorted(atom_features, key=lambda x: x["coordid"])
        num_atoms = mol.OBMol.NumAtoms()
        for i, j in itertools.combinations(range(0, num_atoms), 2):
            bond_feature = self.get_pair_feature_coupling(mol, coupling_attributes, i, j, full_pair_matrix,
                                                          target_convert)
            if bond_feature:
                atom_pairs.append(bond_feature)
            else:
                continue

        # Compute the graph distance, if desired
        if 'graph_distance' in self.bond_features:
            graph_dist = self._dijkstra_distance(atom_pairs)
            for i in atom_pairs:
                i.update({'graph_distance': graph_dist[i['a_idx'], i['b_idx']]})

        # Generate the state attributes (that describe the whole network)
        state_attributes = state_attributes or [
            [mol.molwt / num_atoms,
             len([i for i in atom_pairs if i['bond_type'] > 0]) / num_atoms, energy_dipole['dipole_moments'][0],
             energy_dipole['potential_energy'][0]]
        ]

        # Get the atom features in the order they are requested by the user as a 2D array
        atoms = []
        for atom in atom_features:
            atoms.append(self._create_atom_feature_vector(atom))

        # Get the bond features in the order request by the user
        bonds = []
        index1_temp = []
        index2_temp = []
        scal_coupling_constant = []
        bonds_pair_count = []
        for bond in atom_pairs:
            # Store the index of each bond
            index1_temp.append(bond.pop('a_idx'))
            index2_temp.append(bond.pop('b_idx'))
            # train out data
            if bond.get('bond_index_pair'):
                bonds_pair_count.append(bond.pop('bond_index_pair'))

            scal_coupling_constant.append(bond.pop('scalar_coupling_constant'))
            # Get the desired bond features
            bonds.append(self._create_pair_feature_vector(bond))

        # Given the bonds (i,j), make it so (i,j) == (j, i)
        index1 = index1_temp + index2_temp
        index2 = index2_temp + index1_temp
        bonds = bonds + bonds
        # bonds_train_out = bonds_train_out+bonds_train_out
        scal_coupling_constant = scal_coupling_constant + scal_coupling_constant

        # Sort the arrays by the beginning index
        sorted_arg = np.argsort(index1)
        index1 = np.array(index1)[sorted_arg].tolist()
        index2 = np.array(index2)[sorted_arg].tolist()

        bonds = np.array(bonds)[sorted_arg].tolist()
        # bonds_train_out = np.array(bonds_train_out)[sorted_arg].tolist()
        scal_coupling_constant = np.array(scal_coupling_constant)[sorted_arg].tolist()

        scal_coupling_constant = list(scal_coupling_constant + [0] * (756 - len(scal_coupling_constant)))
        # print("vvvvv")
        # print(scal_coupling_constant.shape)
        # print(scal_coupling_constant)
        # scal_coupling_constant = scal_coupling_constant.reshape(-1,1)
        # print(scal_coupling_constant.shape)

        gnode = [0] * len(atoms)
        gbond = [0] * len(index1)
        if target_convert == False:
            return ({'atom': atoms,
                     'bond': bonds,
                     'state': state_attributes,
                     'index1': index1,
                     'index2': index2

                     }, {'index1': index1,
                         'index2': index2,
                         'coupling_attributes': coupling_attributes
                         })

        return ({'atom': atoms,
                 'bond': bonds,
                 'state': state_attributes,
                 'index1': index1,
                 'index2': index2,

                 }, {'scal_coupling_constant': scal_coupling_constant})

    def convert_coupling(self, mol, coupling_attributes, energy_dipole, charges_magnetic, target_convert=True,
                         state_attributes=None, full_pair_matrix=True):
        """
        Compute the representation for a molecule

        Args：
            mol (pybel.Molecule): Molecule to generate features for
            state_attributes (list): State attributes. Uses average mass and number of bonds per atom as default
            full_pair_matrix (bool): Whether to generate info for all atom pairs, not just bonded ones
        Returns:
            (dict): Dictionary of features
        """

        # Get the features features for all atoms and bonds
        atom_features = []
        atom_pairs = []
        for idx, atom in enumerate(mol.atoms):
            get_data = charges_magnetic[charges_magnetic['atom_index'] == idx]
            f = self.get_atom_feature(mol, get_data, atom)
            atom_features.append(f)
        atom_features = sorted(atom_features, key=lambda x: x["coordid"])
        num_atoms = mol.OBMol.NumAtoms()
        # get the bonds
        # for index,row in coupling_attributes.iterrows():
        #     atom_index_0 = row["atom_index_0"]
        #     atom_index_1 = row['atom_index_1']
        #     type = row['type']
        #     if target_convert:
        #         scalar_coupling_constant = row['scalar_coupling_constant']
        #     else:
        #         scalar_coupling_constant=0
        for i, j in itertools.combinations(range(0, num_atoms), 2):
            bond_feature = self.get_pair_feature_coupling(mol, coupling_attributes, i, j, full_pair_matrix,
                                                          target_convert)
            if bond_feature:
                atom_pairs.append(bond_feature)
            else:
                continue

        # Compute the graph distance, if desired
        if 'graph_distance' in self.bond_features:
            graph_dist = self._dijkstra_distance(atom_pairs)
            for i in atom_pairs:
                i.update({'graph_distance': graph_dist[i['a_idx'], i['b_idx']]})

        # Generate the state attributes (that describe the whole network)
        state_attributes = state_attributes or [
            [mol.molwt / num_atoms,
             len([i for i in atom_pairs if i['bond_type'] > 0]) / num_atoms
           ]
        ]
       ##mol_out
        mol_out=[]

        mol_out.append( energy_dipole['dipole_moments'].tolist()[0])
        mol_out.append(energy_dipole['potential_energy'].tolist()[0])

        # Get the atom features in the order they are requested by the user as a 2D array
        atoms = []
        atmos_out=[]

        for atom in atom_features:

            atmos_out.append(atom.pop('sigma_iso'))
            atmos_out.append(atom.pop('omega'))
            atmos_out.append(atom.pop('kappa'))
            atmos_out.append(atom.pop('mulliken_charge'))

            atoms.append(self._create_atom_feature_vector(atom))

        # Get the bond features in the order request by the user
        bonds = []
        index1_temp = []
        index2_temp = []
        scal_coupling_constant = []
        bonds_pair_count = []
        for bond in atom_pairs:
            # Store the index of each bond
            index1_temp.append(bond.pop('a_idx'))
            index2_temp.append(bond.pop('b_idx'))
            # train out data

            scal_coupling_constant.append(bond.pop('scalar_coupling_constant'))
            # Get the desired bond features
            bonds.append(self._create_pair_feature_vector(bond))

        # Given the bonds (i,j), make it so (i,j) == (j, i)
        index1 = index1_temp + index2_temp
        index2 = index2_temp + index1_temp
        bonds = bonds + bonds
        # bonds_train_out = bonds_train_out+bonds_train_out
        scal_coupling_constant = scal_coupling_constant + scal_coupling_constant

        # Sort the arrays by the beginning index
        sorted_arg = np.argsort(index1)
        index1 = np.array(index1)[sorted_arg].tolist()
        index2 = np.array(index2)[sorted_arg].tolist()

        bonds = np.array(bonds)[sorted_arg].tolist()
        # bonds_train_out = np.array(bonds_train_out)[sorted_arg].tolist()
        scal_coupling_constant = np.array(scal_coupling_constant)[sorted_arg].tolist()

        scal_coupling_constant = list(scal_coupling_constant + [0] * (820 - len(scal_coupling_constant)))

        # atmos_out = list(atmos_out + [0] * (112 - len(atmos_out)))

        # output = np.concatenate([mol_out,atmos_out,scal_coupling_constant],axis=0)

        # print(scal_coupling_constant.shape)
        # print(scal_coupling_constant)
        # scal_coupling_constant = scal_coupling_constant.reshape(-1,1)
        # print(scal_coupling_constant.shape)

        if target_convert == False:
            return ({'atom': atoms,
                     'bond': bonds,
                     'state': state_attributes,
                     'index1': index1,
                     'index2': index2

                     }, {'index1': index1,
                         'index2': index2,
                         'coupling_attributes': coupling_attributes
                         })

        return ({'atom': atoms,
                 'bond': bonds,
                 'state': state_attributes,
                 'index1': index1,
                 'index2': index2

                 }, {'scal_coupling_constant': scal_coupling_constant})

    def _create_pair_feature_vector(self, bond: dict) -> List[float]:
        """Generate the feature vector from the bond feature dictionary

        Handles the binarization of categorical variables, and performing the distance conversion

        Args:
            bond (dict): Features for a certain pair of atoms
        Returns:
            ([float]) Values converted to a vector
            """
        bond_temp = []
        for i in self.bond_features:
            # Some features require conversion (e.g., binarization)
            if i in bond:
                if i == "bond_type":
                    bond_temp.extend(fast_label_binarize(bond[i], [0, 1, 2, 3, 4]))
                elif i == "same_ring":
                    bond_temp.append(int(bond[i]))
                elif i == "coupling_type":
                    bond_temp.extend(fast_label_binarize(bond[i], [0, 1, 2, 3, 4, 5, 6, 7]))
                elif i == "scalar_coupling_constant":
                    bond_temp.append(float(bond[i]))

                elif i == "spatial_distance":
                    expanded = self.distance_converter.convert([bond[i]])[0]
                    if isinstance(expanded, np.ndarray):
                        # If we use a distance expansion
                        bond_temp.extend(expanded.tolist())
                    else:
                        # If not
                        bond_temp.append(expanded)
                else:
                    bond_temp.append(bond[i])
        return bond_temp

    def _create_atom_feature_vector(self, atom: dict) -> List[int]:
        """Generate the feature vector from the atomic feature dictionary

        Handles the binarization of categorical variables, and transforming the ring_sizes to a list

        Args:
            atom (dict): Dictionary of atomic features
        Returns:
            ([int]): Atomic feature vector
        """
        atom_temp = []
        for i in self.atom_features:
            if i == 'chirality':
                atom_temp.extend(fast_label_binarize(atom[i], [0, 1, 2]))
            elif i == 'element':
                atom_temp.extend(fast_label_binarize(atom[i], self.known_elements))
            elif i in ['aromatic', 'donor', 'acceptor']:
                atom_temp.append(int(atom[i]))
            elif i == 'hybridization':
                atom_temp.extend(fast_label_binarize(atom[i], [1, 2, 3, 4, 5, 6]))
            elif i == 'ring_sizes':
                atom_temp.extend(ring_to_vector(atom[i]))
            else:  # It is a scalar
                atom_temp.append(atom[i])
        return atom_temp

    def _dijkstra_distance(self, pairs):
        """
        Compute the graph distance between each pair of atoms,
        using the network defined by the bonded atoms.

        Args:
            pairs ([dict]): List of bond information
        Returns:
            ([int]) Distance for each pair of bonds
        """
        bonds = []
        for p in pairs:
            if p['bond_type'] > 0:
                bonds.append([p['a_idx'], p['b_idx']])

        return dijkstra_distance(bonds)

    def get_atom_feature(self, mol, charges_magnetic, atom):
        """
        Generate all features of a particular atom

        Args:
            mol (pybel.Molecule): Molecule being evaluated
            atom (pybel.Atom): Specific atom being evaluated
        Return:
            (dict): All features for that atom
        """

        # Get the link to the OpenBabel representation of the atom
        obatom = atom.OBAtom
        atom_idx = atom.idx - 1  # (pybel atoms indices start from 1)

        # Get the element
        element = Element.from_Z(obatom.GetAtomicNum()).symbol

        # Get the fast-to-compute properties
        output = {"element": element,
                  "atomic_num": obatom.GetAtomicNum(),
                  "formal_charge": obatom.GetFormalCharge(),
                  "hybridization": 6 if element == 'H' else obatom.GetHyb(),
                  "acceptor": obatom.IsHbondAcceptor(),
                  "donor": obatom.IsHbondDonorH() if atom.type == 'H' else obatom.IsHbondDonor(),
                  "aromatic": obatom.IsAromatic(),
                  "coordid": atom.coordidx,
                  "sigma_iso": charges_magnetic['sigma_iso'].tolist()[0],
                  "omega": charges_magnetic['omega'].tolist()[0],
                  "kappa": charges_magnetic['kappa'].tolist()[0],
                  "mulliken_charge": charges_magnetic['mulliken_charge'].tolist()[0]

                  }

        # Get the chirality, if desired
        if 'chirality' in self.atom_features:
            # Determine whether the molecule has chiral centers
            chiral_cc = self._get_chiral_centers(mol)
            if atom_idx not in chiral_cc:
                output['chirality'] = 0
            else:
                # 1 --> 'R', 2 --> 'S'
                output['chirality'] = 1 if chiral_cc[atom_idx] == 'R' else 2

        # Find the rings, if desired
        if 'ring_sizes' in self.atom_features:
            rings = mol.OBMol.GetSSSR()  # OpenBabel caches ring computation internally, no need to cache ourselves
            output['ring_sizes'] = [r.Size() for r in rings if r.IsInRing(atom.idx)]

        return output

    def create_bond_feature(self, mol, bid, eid):
        """
        Create information for a bond for a pair of atoms that are not actually bonded

        Args:
            mol (pybel.Molecule): Molecule being featurized
            bid (int): Index of atom beginning of the bond
            eid (int): Index of atom at the end of the bond
        """
        a1 = mol.OBMol.GetAtom(bid + 1)
        a2 = mol.OBMol.GetAtom(eid + 1)
        same_ring = mol.OBMol.AreInSameRing(a1, a2)
        return {"a_idx": bid,
                "b_idx": eid,
                "bond_type": 0,
                "same_ring": True if same_ring else False,
                "spatial_distance": a1.GetDistance(a2)}

    def create_bond_feature_coupling(self, mol, bid, eid, coupling_type, scalar_coupling_constant, bond_index_pair):
        """
        Create information for a bond for a pair of atoms that are not actually bonded

        Args:
            mol (pybel.Molecule): Molecule being featurized
            bid (int): Index of atom beginning of the bond
            eid (int): Index of atom at the end of the bond
        """
        a1 = mol.OBMol.GetAtom(bid + 1)
        a2 = mol.OBMol.GetAtom(eid + 1)
        same_ring = mol.OBMol.AreInSameRing(a1, a2)
        return {"a_idx": bid,
                "b_idx": eid,
                "bond_type": 0,
                "same_ring": True if same_ring else False,
                "spatial_distance": a1.GetDistance(a2),
                "coupling_type": coupling_type,
                "scalar_coupling_constant": scalar_coupling_constant,
                "bond_index_pair": bond_index_pair
                }

    def get_pair_feature(self, mol, bid, eid, full_pair_matrix):
        """
        Get the features for a certain bond

        Args:
            mol (pybel.Molecule): Molecule being featurized
            bid (int): Index of atom beginning of the bond
            eid (int): Index of atom at the end of the bond
            full_pair_matrix (bool): Whether to compute the matrix for every atom - even those that
                are not actually bonded
        """
        # Find the bonded pair of atoms
        bond = mol.OBMol.GetBond(bid + 1, eid + 1)
        if not bond:  # If the bond is ordered in the other direction
            bond = mol.OBMol.GetBond(eid + 1, bid + 1)

        # If the atoms are not bonded
        if not bond:
            if full_pair_matrix:
                return self.create_bond_feature(mol, bid, eid)
            else:
                return None

        # Compute bond features
        a1 = mol.OBMol.GetAtom(bid + 1)
        a2 = mol.OBMol.GetAtom(eid + 1)
        same_ring = mol.OBMol.AreInSameRing(a1, a2)
        return {"a_idx": bid,
                "b_idx": eid,
                "bond_type": 4 if bond.IsAromatic() else bond.GetBondOrder(),
                "same_ring": True if same_ring else False,
                "spatial_distance": a1.GetDistance(a2)}

    def get_pair_feature_coupling1(self, mol, type, bid, eid, full_pair_matrix, target_convert=False,
                                   scalar_coupling_constant=0):
        """
        Get the features for a certain bond

        Args:
            mol (pybel.Molecule): Molecule being featurized
            bid (int): Index of atom beginning of the bond
            eid (int): Index of atom at the end of the bond
            full_pair_matrix (bool): Whether to compute the matrix for every atom - even those that
                are not actually bonded
        """
        # print(coupling_attr.columns)

        # coupling_arr0 = coupling_attr[(coupling_attr['atom_index_0']==bid) & (coupling_attr['atom_index_1']==eid)]
        # coupling_arr1 = coupling_attr[(coupling_attr['atom_index_0'] == eid) & (coupling_attr['atom_index_1'] == bid)]
        #
        # # print(bid,eid)
        #
        # coupling_type=-1
        # scalar_coupling_constant=0
        # bond_index_pair=[]
        # if(len(coupling_arr0)>0):
        #     coupling_type = coupling_arr0['type'].tolist()[0]
        #     if(target_convert):
        #         scalar_coupling_constant = coupling_arr0['scalar_coupling_constant'].tolist()[0]
        #     bond_index_pair.append((bid,eid))
        #     # print("ddddd")
        #     # print(bond_index_pair)
        # if (len(coupling_arr1) > 0):
        #     coupling_type = coupling_arr1['type'].tolist()[0]
        #     if (target_convert):
        #         scalar_coupling_constant = coupling_arr1['scalar_coupling_constant'].tolist()[0]
        #     bond_index_pair.append((eid, bid))
        # print("fffff")
        # print(bond_index_pair)

        # print(coupling_arr)
        ##get coupling attr

        # Find the bonded pair of atoms
        bond = mol.OBMol.GetBond(bid + 1, eid + 1)
        if not bond:  # If the bond is ordered in the other direction
            bond = mol.OBMol.GetBond(eid + 1, bid + 1)

        # If the atoms are not bonded
        if not bond:
            if full_pair_matrix:
                return self.create_bond_feature_coupling(mol, bid, eid, type, scalar_coupling_constant)
            else:
                return None

        # Compute bond features
        a1 = mol.OBMol.GetAtom(bid + 1)
        a2 = mol.OBMol.GetAtom(eid + 1)
        same_ring = mol.OBMol.AreInSameRing(a1, a2)

        if (target_convert == False):
            scalar_coupling_constant = 0
        # else:
        #     scalar_coupling_constant = coupling_arr1['scalar_coupling_constant'].tolist()[0]
        return {"a_idx": bid,
                "b_idx": eid,
                "bond_type": 4 if bond.IsAromatic() else bond.GetBondOrder(),
                "same_ring": True if same_ring else False,
                "spatial_distance": a1.GetDistance(a2),
                "coupling_type": type,
                "scalar_coupling_constant": scalar_coupling_constant,
                }

    def get_pair_feature_coupling(self, mol, coupling_attr, bid, eid, full_pair_matrix, target_convert=False):
        """
        Get the features for a certain bond

        Args:
            mol (pybel.Molecule): Molecule being featurized
            bid (int): Index of atom beginning of the bond
            eid (int): Index of atom at the end of the bond
            full_pair_matrix (bool): Whether to compute the matrix for every atom - even those that
                are not actually bonded
        """
        # print(coupling_attr.columns)
        coupling_arr0 = coupling_attr[(coupling_attr['atom_index_0'] == bid) & (coupling_attr['atom_index_1'] == eid)]
        coupling_arr1 = coupling_attr[(coupling_attr['atom_index_0'] == eid) & (coupling_attr['atom_index_1'] == bid)]

        # print(bid,eid)

        coupling_type = -1
        scalar_coupling_constant = 0
        bond_index_pair = []
        if (len(coupling_arr0) > 0):
            coupling_type = coupling_arr0['type'].tolist()[0]
            if (target_convert):
                scalar_coupling_constant = coupling_arr0['scalar_coupling_constant'].tolist()[0]
            bond_index_pair.append((bid, eid))
            # print("ddddd")
            # print(bond_index_pair)
        if (len(coupling_arr1) > 0):
            coupling_type = coupling_arr1['type'].tolist()[0]
            if (target_convert):
                scalar_coupling_constant = coupling_arr1['scalar_coupling_constant'].tolist()[0]
            bond_index_pair.append((eid, bid))
            # print("fffff")
            # print(bond_index_pair)

        # print(coupling_arr)
        ##get coupling attr

        # Find the bonded pair of atoms
        bond = mol.OBMol.GetBond(bid + 1, eid + 1)
        if not bond:  # If the bond is ordered in the other direction
            bond = mol.OBMol.GetBond(eid + 1, bid + 1)

        # If the atoms are not bonded
        if not bond:
            if full_pair_matrix:
                return self.create_bond_feature_coupling(mol, bid, eid, coupling_type, scalar_coupling_constant,
                                                         bond_index_pair)
            else:
                return None

        # Compute bond features
        a1 = mol.OBMol.GetAtom(bid + 1)
        a2 = mol.OBMol.GetAtom(eid + 1)
        same_ring = mol.OBMol.AreInSameRing(a1, a2)

        if (target_convert == False):
            scalar_coupling_constant = 0
        # else:
        #     scalar_coupling_constant = coupling_arr1['scalar_coupling_constant'].tolist()[0]
        return {"a_idx": bid,
                "b_idx": eid,
                "bond_type": 4 if bond.IsAromatic() else bond.GetBondOrder(),
                "same_ring": True if same_ring else False,
                "spatial_distance": a1.GetDistance(a2),
                "coupling_type": coupling_type,
                "scalar_coupling_constant": scalar_coupling_constant,
                "bond_index_pair": bond_index_pair
                }

    def _get_rdk_mol(self, mol, format='smiles'):
        """
        Return: RDKit Mol (w/o H)
        """
        if format == 'pdb':
            return Chem.rdmolfiles.MolFromPDBBlock(mol.write("pdb"))
        elif format == 'smiles':
            return Chem.rdmolfiles.MolFromSmiles(mol.write("smiles"))

    def _get_chiral_centers(self, mol):
        """
        Use RDKit to find the chiral centers with CIP(R/S) label

        This provides the absolute stereochemistry.  The chiral label obtained
        from pybabel and rdkit.mol.getchiraltag is relative positions of the bonds as provided

        Args:
            mol (Molecule): Molecule to asses
        Return:
            (dict): Keys are the atom index and values are the CIP label
        """
        mol_rdk = self._get_rdk_mol(mol, 'smiles')
        if mol_rdk is None:
            # Conversion to RDKit has failed
            return {}
        else:
            chiral_cc = Chem.FindMolChiralCenters(mol_rdk)
            return dict(chiral_cc)


def dijkstra_distance(bonds):
    """
    Compute the graph distance based on the dijkstra algorithm

    Args:
        bonds: (list of list), for example [[0, 1], [1, 2]] means two bonds formed by atom 0, 1 and atom 1, 2

    Returns:
        full graph distance matrix
    """
    nb_atom = max(itertools.chain(*bonds)) + 1
    graph_dist = np.ones((nb_atom, nb_atom), dtype=np.int32) * np.infty
    for bond in bonds:
        graph_dist[bond[0], bond[1]] = 1
        graph_dist[bond[1], bond[0]] = 1

    queue = deque()  # Queue used in all loops
    visited = set()  # Used in all loops
    for i in range(nb_atom):
        graph_dist[i, i] = 0
        visited.clear()
        queue.append(i)
        while queue:
            s = queue.pop()
            visited.add(s)

            for k in np.where(graph_dist[s, :] == 1)[0]:
                if k not in visited:
                    queue.append(k)
                    graph_dist[i, k] = min(graph_dist[i, k],
                                           graph_dist[i, s] + 1)
                    graph_dist[k, i] = graph_dist[i, k]
    return graph_dist


def mol_from_smiles(smiles):
    mol = pybel.readstring(format='smi', string=smiles)
    mol.make3D()
    return mol


def mol_from_pymatgen(mol):
    """
    Args:
        mol(Molecule)
    """
    mol = pybel.Molecule(BabelMolAdaptor(mol).openbabel_mol)
    mol.make3D()
    return mol


def mol_from_file(file_path, file_format='xyz'):
    """
    Args:
        file_path(str)
        file_format(str): allow formats that open babel supports
    """
    mol = [r for r in pybel.readfile(format=file_format,
                                     filename=file_path)][0]
    return mol


def _convert_mol(mol, molecule_format, converter):
    """Convert a molecule from string to its graph features

    Utility function used in the graph generator.

    The parse and convert operations are both in this function due to Pybel objects
    not being serializable. By not using the Pybel representation of each molecule
    as an input to this function, we can use multiprocessing to parallelize conversion
    over molecules as strings can be passed as pickle objects to the worker threads but
    but Pybel objects cannot.

    Args:
        mol (str): String representation of a molecule
        molecule_format (str): Format of the string representation
        converter (MolecularGraph): Tool used to generate graph representation
    Returns:
        (dict): Graph representation of the molecule
    """

    # Convert molecule into pybel format
    if molecule_format == 'smiles':
        mol = mol_from_smiles(mol)  # Used to generate 3D coordinates/H atoms
    else:
        mol = pybel.readstring(molecule_format, mol)

    return converter.convert(mol)


class MolecularGraphBatchGenerator(BaseGraphBatchGenerator):
    """Generator that creates batches of molecular data by computing graph properties on demand

    If your dataset is small enough that the descriptions of the whole dataset fit in memory,
    we recommend using :class:`megnet.data.graph.GraphBatchGenerator` instead to avoid
    the computational cost of dynamically computing graphs."""

    def __init__(self, mols, targets=None, converter=None, molecule_format='xyz',
                 batch_size=128, shuffle=True, n_jobs=1):
        """
        Args:
            mols ([str]): List of the string reprensetations of each molecule
            targets ([ndarray]): Properties of each molecule to be predicted
            converter (MolecularGraph): Converter used to generate graph features
            molecule_format (str): Format of each of the string representations in `mols`
            batch_size (int): Target size for each batch
            shuffle (bool): Whether to shuffle the training data after each epoch
            n_jobs (int): Number of worker threads (None to use all threads).
        """

        super().__init__(len(mols), targets, batch_size, shuffle)
        self.mols = np.array(mols)
        if converter is None:
            converter = MolecularGraph()
        self.converter = converter
        self.molecule_format = molecule_format
        self.n_jobs = n_jobs

        def mute():
            sys.stdout = open(os.devnull, 'w')
            sys.stderr = open(os.devnull, 'w')

        self.pool = Pool(self.n_jobs, initializer=mute) if self.n_jobs != 1 else None

    def __del__(self):
        if self.pool is not None:
            self.pool.close()  # Kill thread pool if generator is deleted

    def _generate_inputs(self, batch_index):
        # Get the molecules for this batch
        mols = self.mols[batch_index]

        # Generate the graphs
        graphs = self._generate_graphs(mols)

        # Return them as flattened into array format
        return self.converter.get_flat_data(graphs)

    def _generate_graphs(self, mols):
        """Generate graphs for a certain collection of molecules

        Args:
            mols ([string]): Molecules to process
        Returns:
            ([dict]): Graphs for all of the molecules
        """
        if self.pool is None:
            graphs = [_convert_mol(m, self.molecule_format, self.converter) for m in mols]
        else:
            func = partial(_convert_mol, molecule_format=self.molecule_format,
                           converter=self.converter)
            graphs = self.pool.map(func, mols)
        return graphs

    def create_cached_generator(self) -> GraphBatchGenerator:
        """Generates features for all of the molecules and stores them in memory

        Returns:
            (GraphBatchGenerator) Graph genereator that relies on having the graphs in memory
        """

        # Make all the graphs
        graphs = self._generate_graphs(self.mols)

        # Turn them into a fat array
        inputs = self.converter.get_flat_data(graphs, self.targets)

        return GraphBatchGenerator(*inputs, is_shuffle=self.is_shuffle,
                                   batch_size=self.batch_size)


class GraphModel:
    """
    Composition of keras model and converter class for transfering structure
    object to input tensors. We add methods to train the model from
    (structures, targets) pairs

    Args:
        model: (keras model)
        graph_converter: (object) a object that turns a structure to a graph,
            check `megnet.data.crystal`
        target_scaler: (object) a scaler object for converting targets, check
            `megnet.utils.preprocessing`
        metadata: (dict) An optional dict of metadata associated with the model.
            Recommended to incorporate some basic information such as units,
            MAE performance, etc.

    """

    def __init__(self,
                 model,
                 graph_converter,
                 target_scaler=DummyScaler(),
                 metadata=None,
                 **kwargs):
        self.model = model
        self.graph_converter = graph_converter
        self.target_scaler = target_scaler
        self.metadata = metadata or {}

    def __getattr__(self, p):
        return getattr(self.model, p)

    def train(self,
              train_structures,
              train_targets,
              validation_structures=None,
              validation_targets=None,
              epochs=1000,
              batch_size=128,
              verbose=1,
              callbacks=None,
              scrub_failed_structures=False,
              prev_model=None,
              lr_scaling_factor=0.5,
              patience=500,
              **kwargs):
        """
        Args:
            train_structures: (list) list of pymatgen structures
            train_targets: (list) list of target values
            validation_structures: (list) list of pymatgen structures as validation
            validation_targets: (list) list of validation targets
            epochs: (int) number of epochs
            batch_size: (int) training batch size
            verbose: (int) keras fit verbose, 0 no progress bar, 1 only at the epoch end and 2 every batch
            callbacks: (list) megnet or keras callback functions for training
            scrub_failed_structures: (bool) whether to scrub structures with failed graph computation
            prev_model: (str) file name for previously saved model
            lr_scaling_factor: (float, less than 1) scale the learning rate down when nan loss encountered
            patience: (int) patience for early stopping
            **kwargs:
        """
        train_graphs, train_targets = self.get_all_graphs_targets(train_structures, train_targets,
                                                                  scrub_failed_structures=scrub_failed_structures)
        if validation_structures is not None:
            val_graphs, validation_targets = self.get_all_graphs_targets(
                validation_structures, validation_targets, scrub_failed_structures=scrub_failed_structures)
        else:
            val_graphs = None

        self.train_from_graphs(train_graphs,
                               train_targets,
                               validation_graphs=val_graphs,
                               validation_targets=validation_targets,
                               epochs=epochs,
                               batch_size=batch_size,
                               verbose=verbose,
                               callbacks=callbacks,
                               prev_model=prev_model,
                               lr_scaling_factor=lr_scaling_factor,
                               patience=patience,
                               **kwargs
                               )

    def train_coupling(self,
                       mols,
                       validation_structures=None,
                       validation_targets=None,
                       epochs=1000,
                       batch_size=128,
                       verbose=1,
                       callbacks=None,
                       scrub_failed_structures=False,
                       prev_model=None,
                       lr_scaling_factor=0.5,
                       patience=500,
                       **kwargs):
        """
        Args:
            train_structures: (list) list of pymatgen structures
            train_targets: (list) list of target values
            validation_structures: (list) list of pymatgen structures as validation
            validation_targets: (list) list of validation targets
            epochs: (int) number of epochs
            batch_size: (int) training batch size
            verbose: (int) keras fit verbose, 0 no progress bar, 1 only at the epoch end and 2 every batch
            callbacks: (list) megnet or keras callback functions for training
            scrub_failed_structures: (bool) whether to scrub structures with failed graph computation
            prev_model: (str) file name for previously saved model
            lr_scaling_factor: (float, less than 1) scale the learning rate down when nan loss encountered
            patience: (int) patience for early stopping
            **kwargs:
        """

#         train_graphs, train_targets = self.get_all_graphs_targets_coupling(mols,
#                                                                           scrub_failed_structures=scrub_failed_structures)

        data_graphs=[]
        data_targets=[]

        with open('../input/shiyan-graph/train_graphs', 'rb') as fp:
            data_graphs = pickle.load(fp)

        with open('../input/shiyan-graph/train_targets', 'rb') as fp:
            data_targets = pickle.load(fp)

#         with open('../input/shiyan-graph-1/train_graphs', 'rb') as fp:
#             data_graphs = pickle.load(fp)

#         with open('../input/shiyan-graph-1/train_targets', 'rb') as fp:
#             data_targets = pickle.load(fp)
        
#         max_value =204.88 
#         min_value = -36.2186

#         for target in  data_targets:
#             target['output'] =  (np.array(target['output'])-min_value)/(max_value-min_value) 
#         data_targets[0]
# #         print(len(data_targets))
#         print(data_targets[0])
        train_graphs, val_graphs, train_targets, validation_targets = train_test_split(data_graphs, data_targets,
                                                                                       test_size=0.20, random_state=42)

#         if validation_structures is not None:
#             val_graphs, validation_targets = self.get_all_graphs_targets_coupling(
#                 mols, scrub_failed_structures=scrub_failed_structures)
#         else:
#             val_graphs = None

        self.train_from_graphs_coupling(train_graphs,
                                        train_targets,
                                        validation_graphs=val_graphs,
                                        validation_targets=validation_targets,
                                        epochs=epochs,
                                        batch_size=batch_size,
                                        verbose=verbose,
                                        callbacks=callbacks,
                                        prev_model=prev_model,
                                        lr_scaling_factor=lr_scaling_factor,
                                        patience=patience,
                                        **kwargs
                                        )

    def train_from_graphs_coupling(self,
                                   train_graphs,
                                   train_targets,
                                   validation_graphs=None,
                                   validation_targets=None,
                                   epochs=1000,
                                   batch_size=1,
                                   verbose=2,
                                   callbacks=None,
                                   prev_model=None,
                                   lr_scaling_factor=0.5,
                                   patience=500,
                                   **kwargs
                                   ):

        # load from saved model
        if prev_model:
            self.load_weights(prev_model)
        is_classification = False
        monitor = 'val_acc' if is_classification else 'val_loss'
        mode = 'max' if is_classification else 'min'
        dirname = kwargs.pop('dirname', 'callback')
        if not os.path.isdir(dirname):
            os.makedirs(dirname)
        if callbacks is None:
            filepath = "weights.best.hdf5"
            checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,
                                         mode='min')
#             stop = keras.callbacks.EarlyStopping( monitor='val_loss', patience=0, verbose=0, mode='auto')

            callbacks= [checkpoint,stop_early,lrate]
        train_nb_atoms = [len(i['atom']) for i in train_graphs]
        train_targets = [self.target_scaler.transform(i, j) for i, j in zip(train_targets, train_nb_atoms)]

        if validation_graphs is not None:
            filepath = os.path.join(dirname, '%s_{epoch:05d}_{%s:.6f}.hdf5' % (monitor, monitor))
            val_nb_atoms = [len(i['atom']) for i in validation_graphs]
            validation_targets = [self.target_scaler.transform(i, j) for i, j in zip(validation_targets, val_nb_atoms)]
            val_inputs = self.graph_converter.get_flat_scarlar_data(validation_graphs, validation_targets)

            val_generator = self._create_generator(*val_inputs,
                                                   batch_size=batch_size)
            steps_per_val = int(np.ceil(len(validation_graphs) / batch_size))
#             callbacks.extend([ReduceLRUponNan(filepath=filepath,
#                                               monitor=monitor,
#                                               mode=mode,
#                                               factor=lr_scaling_factor,
#                                               patience=patience,
#                                               )])
#             callbacks.extend([ModelCheckpointMAE(filepath=filepath,
#                                                  monitor=monitor,
#                                                  mode=mode,
#                                                  save_best_only=True,
#                                                  save_weights_only=False,
#                                                  val_gen=val_generator,
#                                                  steps_per_val=steps_per_val,
#                                                  target_scaler=self.target_scaler)])
        else:
            val_generator = None
            steps_per_val = None
        # print(np.array(train_graphs[2]).shape)
        train_inputs = self.graph_converter.get_flat_scarlar_data(train_graphs, train_targets)

        # print(np.array(train_inputs[1]).shape)
        # print(np.array(train_inputs[5]).shape)
        # check dimension match
        # print(np.array(train_graphs).shape)
        self.check_dimension(train_graphs[0])
        # ########################################new dynamics code####################
        #         target_ph = tf.placeholder(tf.float32, shape=(None, 1))
        #         target_data = tf.data.Dataset.from_tensor_slices(target_ph)
        #         target_data = target_data.batch(batch_size)
        #         target_iter = target_data.make_initializable_iterator()
        #         target = target_iter.get_next()
        #         learning_rate = 1e-3  # Learning rate
        #
        #         optimizer = Adam(lr=learning_rate)
        #         self.compile(optimizer=optimizer, loss='mse', target_tensors=target)
        #
        #         # Training setup
        #         sess = K.get_session()
        #         print(batch_size)
        #
        #         # input = [train_inputs]
        #         batches_train = batch_iterator([train_inputs[0],train_inputs[1],train_inputs[2],train_inputs[3],train_inputs[4],train_inputs[5],train_inputs[6],train_inputs[7]], 1, epochs)
        #         loss = 0
        #         batch_index = 0
        #         batches_in_epoch = np.ceil(len(train_graphs[0]) / batch_size)
        #
        #         # Training loop
        #         for b in batches_train:
        #             # batch = Batch(b[0], b[1],b[2],b[3],b[4],b[5],b[6])
        #             y_ = b[7]
        #             print(y_[0])
        #             print(y_[0].shape)
        #             sess.run(target_iter.initializer, feed_dict={target_ph: y_})
        #             loss += self.train_on_batch([b[0], b[1],b[2],b[3],b[4],b[5],b[6]], None)
        #
        #             batch_index += 1
        #             if batch_index == batches_in_epoch:
        #                 print(batch_index)
        #                 print('Loss: {}'.format(loss / batches_in_epoch))
        #                 loss = 0
        #                 # batch_index = 0
        #         print(batch_index)

        train_generator = self._create_generator(*train_inputs, batch_size=batch_size)
        steps_per_train = int(np.ceil(len(train_graphs) / batch_size))
        self.fit_generator(train_generator, steps_per_epoch=steps_per_train,
                           validation_data=val_generator, validation_steps=steps_per_val,
                           epochs=epochs, initial_epoch = 0, verbose=verbose, callbacks=callbacks, **kwargs)

    def train_from_graphs(self,
                          train_graphs,
                          train_targets,
                          validation_graphs=None,
                          validation_targets=None,
                          epochs=1000,
                          batch_size=128,
                          verbose=1,
                          callbacks=None,
                          prev_model=None,
                          lr_scaling_factor=0.5,
                          patience=500,
                          **kwargs
                          ):

        # load from saved model
        if prev_model:
            self.load_weights(prev_model)
        is_classification = 'entropy' in self.model.loss
        monitor = 'val_acc' if is_classification else 'val_mae'
        mode = 'max' if is_classification else 'min'
        dirname = kwargs.pop('dirname', 'callback')
        if not os.path.isdir(dirname):
            os.makedirs(dirname)
        if callbacks is None:
            # with this call back you can stop the model training by `touch STOP`
            keras.callbacks.EarlyStopping(monitor=monitor, patience=0, verbose=0, mode='auto')

            callbacks = [ManualStop(),EarlyStopping]
        train_nb_atoms = [len(i['atom']) for i in train_graphs]
        train_targets = [self.target_scaler.transform(i, j) for i, j in zip(train_targets, train_nb_atoms)]

        if validation_graphs is not None:
            filepath = os.path.join(dirname, '%s_{epoch:05d}_{%s:.6f}.hdf5' % (monitor, monitor))
            val_nb_atoms = [len(i['atom']) for i in validation_graphs]
            validation_targets = [self.target_scaler.transform(i, j) for i, j in
                                  zip(validation_targets, val_nb_atoms)]
            val_inputs = self.graph_converter.get_flat_data(validation_graphs, validation_targets)

            val_generator = self._create_generator(*val_inputs,
                                                   batch_size=batch_size)
            steps_per_val = int(np.ceil(len(validation_graphs) / batch_size))
            callbacks.extend([ReduceLRUponNan(filepath=filepath,
                                              monitor=monitor,
                                              mode=mode,
                                              factor=lr_scaling_factor,
                                              patience=patience,
                                              )])
            callbacks.extend([ModelCheckpointMAE(filepath=filepath,
                                                 monitor=monitor,
                                                 mode=mode,
                                                 save_best_only=True,
                                                 save_weights_only=False,
                                                 val_gen=val_generator,
                                                 steps_per_val=steps_per_val,
                                                 target_scaler=self.target_scaler)])
        else:
            val_generator = None
            steps_per_val = None

        train_inputs = self.graph_converter.get_flat_data(train_graphs, train_targets)
        # check dimension match
        self.check_dimension(train_graphs[0])
        train_generator = self._create_generator(*train_inputs, batch_size=batch_size)
        steps_per_train = int(np.ceil(len(train_graphs) / batch_size))
        self.fit_generator(train_generator, steps_per_epoch=steps_per_train,
                           validation_data=val_generator, validation_steps=steps_per_val,
                           epochs=epochs, verbose=verbose, callbacks=callbacks, **kwargs)

    def check_dimension(self, graph):
        """
        Check the model dimension against the graph converter dimension
        Args:
            graph: structure graph

        Returns:

        """
        test_inp = self.graph_converter.graph_to_input(graph)
        input_shapes = [i.shape for i in test_inp]

        model_input_shapes = [int_shape(i) for i in self.model.inputs]

        def _check_match(real_shape, tensor_shape):
            if len(real_shape) != len(tensor_shape):
                return False
            matched = True
            for i, j in zip(real_shape, tensor_shape):
                if j is None:
                    continue
                else:
                    if i == j:
                        continue
                    else:
                        matched = False
            return matched

        for i, j, k in zip(['atom features', 'bond features', 'state features'],
                           input_shapes[:3], model_input_shapes[:3]):
            matched = _check_match(j, k)
            if not matched:
                raise ValueError("The data dimension for %s is %s and does not match model "
                                 "required shape of %s" % (i, str(j), str(k)))

    def get_all_graphs_targets(self, structures, targets, scrub_failed_structures=False):
        """
        Compute the graphs from structures and spit out (graphs, targets) with options to
        automatically remove structures with failed graph computations

        Args:
            structures: (list) pymatgen structure list
            targets: (list) target property list
            scrub_failed_structures: (bool) whether to scrub those failed structures

        Returns:
            graphs, targets

        """
        graphs_valid = []
        targets_valid = []

        for i, (s, t) in enumerate(zip(structures, targets)):
            try:
                graph = self.graph_converter.convert(s, t)
                graphs_valid.append(graph)
                targets_valid.append(t)
            except Exception as e:
                if scrub_failed_structures:
                    warn("structure with index %d failed the graph computations" % i,
                         UserWarning)
                    continue
                else:
                    raise RuntimeError(str(e))
        return graphs_valid, targets_valid

    def get_all_graphs_targets_coupling(self, mols, scrub_failed_structures=False, target_convert=True):
        """
        Compute the graphs from structures and spit out (graphs, targets) with options to
        automatically remove structures with failed graph computations

        Args:
            structures: (list) pymatgen structure list
            targets: (list) target property list
            scrub_failed_structures: (bool) whether to scrub those failed structures

        Returns:
            graphs, targets

        """
        graphs_valid = []
        targets_valid = []
        # target = np.zeros((len(mols), 135), dtype=np.float32)

        with tqdm(total=len(mols)) as pbar:
            for i, (s, t, energy_dipole, charges_magnetic) in enumerate(mols):
                try:
                    mol = read_ob_molecule(s)
                    mol = pybel.Molecule(mol)
                    graph, train = self.graph_converter.convert_coupling(mol, t, energy_dipole, charges_magnetic,
                                                                         target_convert)
                    # self.check_dimension(graph)
                    # graph = self.graph_converter.graph_to_input(graph)
                    graphs_valid.append(graph)

                    #
                    # # Get the coupling constant data in the order request by the user
                    # train_coupling = np.array(t).tolist()
                    # # train_coupling= t
                    # # bonds = []
                    # index1_temp = []
                    # index2_temp = []
                    # for train in train_coupling:
                    #     # Store the index of each bond
                    #     index1_temp.append(train[2])
                    #     index2_temp.append(train[3])
                    #
                    #
                    # # Given the train (i,j), make it so (i,j) == (j, i)
                    # index1 = index1_temp + index2_temp
                    # index2 = index2_temp + index1_temp
                    # train_coupling = train_coupling + train_coupling
                    #
                    # # Sort the arrays by the beginning index
                    # sorted_arg = np.argsort(index1)
                    # index1 = np.array(index1)[sorted_arg].tolist()
                    # index2 = np.array(index2)[sorted_arg].tolist()
                    # train = np.array(train_coupling)[sorted_arg].tolist()
                    # train_dict = {
                    # 'train': train,
                    # 'index1': index1,
                    # 'index2': index2
                    # }
                    # # target_graph = self.graph_converter.convert(s, t,True)
                    targets_valid.append(train)
                except Exception as e:
                    if scrub_failed_structures:
                        warn("structure with index %d failed the graph computations" % i,
                             UserWarning)
                        continue
                    else:
                        raise RuntimeError(str(e))
                pbar.update()
        return graphs_valid, targets_valid

    def predict_structure(self, structure):
        """
        Predict property from structure

        Args:
            structure: pymatgen structure or molecule

        Returns:
            predicted target value
        """
        graph = self.graph_converter.convert(structure)
        return self.predict_graph(graph)

    def predict_graph(self, graph):
        """
        Predict property from graph

        Args:
            graph: a graph dictionary, see megnet.data.graph

        Returns:
            predicted target value

        """
        inp = self.graph_converter.graph_to_input(graph)
        return self.target_scaler.inverse_transform(self.predict(inp).ravel(), len(graph['atom']))

    def _create_generator(self, *args, **kwargs):
        if hasattr(self.graph_converter, 'bond_converter'):
            kwargs.update({'distance_converter': self.graph_converter.bond_converter})
            return GraphBatchDistanceConvert(*args, **kwargs)
        else:
            return GraphBatchGenerator(*args, **kwargs)

    def save_model(self, filename):
        """
        Save the model to a keras model hdf5 and a json config for additional
        converters

        Args:
            filename: (str) output file name

        Returns:
            None
        """
        self.model.save(filename)
        dumpfn(
            {
                'graph_converter': self.graph_converter,
                'target_scaler': self.target_scaler,
                'metadata': self.metadata
            },
            filename + '.json'
        )

    @classmethod
    def from_file(cls, filename):
        """
        Class method to load model from
            filename for keras model
            filename.json for additional converters

        Args:
            filename: (str) model file name

        Returns
            GraphModel
        """
        configs = loadfn(filename + '.json')
        from keras.models import load_model
        from megnet.layers import _CUSTOM_OBJECTS
        model = load_model(filename, custom_objects=_CUSTOM_OBJECTS)
        configs.update({'model': model})
        return GraphModel(**configs)

    @classmethod
    def from_url(cls, url):
        """
        Download and load a model from a URL. E.g.
        https://github.com/materialsvirtuallab/megnet/blob/master/mvl_models/mp-2019.4.1/formation_energy.hdf5

        Args:
            url: (str) url link of the model

        Returns:
            GraphModel
        """
        import urllib.request
        fname = url.split("/")[-1]
        urllib.request.urlretrieve(url, fname)
        urllib.request.urlretrieve(url + ".json", fname + ".json")
        return cls.from_file(fname)


class MEGNetModel(GraphModel):
    """
    Construct a graph network model with or without explicit atom features
    if n_feature is specified then a general graph model is assumed,
    otherwise a crystal graph model with z number as atom feature is assumed.

    Args:
        nfeat_edge: (int) number of bond features
        nfeat_global: (int) number of state features
        nfeat_node: (int) number of atom features
        nblocks: (int) number of MEGNetLayer blocks
        lr: (float) learning rate
        n1: (int) number of hidden units in layer 1 in MEGNetLayer
        n2: (int) number of hidden units in layer 2 in MEGNetLayer
        n3: (int) number of hidden units in layer 3 in MEGNetLayer
        nvocal: (int) number of total element
        embedding_dim: (int) number of embedding dimension
        nbvocal: (int) number of bond types if bond attributes are types
        bond_embedding_dim: (int) number of bond embedding dimension
        ngvocal: (int) number of global types if global attributes are types
        global_embedding_dim: (int) number of global embedding dimension
        npass: (int) number of recurrent steps in Set2Set layer
        ntarget: (int) number of output targets
        act: (object) activation function
        l2_coef: (float or None) l2 regularization parameter
        is_classification: (bool) whether it is a classification task
        loss: (object or str) loss function
        metrics: (list or dict) List or dictionary of Keras metrics to be evaluated by the model during training and testing
        dropout: (float) dropout rate
        graph_converter: (object) object that exposes a "convert" method for structure to graph conversion
        target_scaler: (object) object that exposes a "transform" and "inverse_transform" methods for transforming the target values
        optimizer_kwargs (dict): extra keywords for optimizer, for example clipnorm and clipvalue
    """

    def __init__(self,
                 nfeat_edge=None,
                 nfeat_global=None,
                 nfeat_node=None,
                 nblocks=3,
                 lr=1e-3,
                 n1=64,
                 n2=32,
                 n3=16,
                 nvocal=95,
                 embedding_dim=16,
                 nbvocal=None,
                 bond_embedding_dim=None,
                 ngvocal=None,
                 global_embedding_dim=None,
                 npass=3,
                 ntarget=1,
                 act=softplus2,
                 is_classification=False,
                 loss="mae",
                 metrics=None,
                 l2_coef=None,
                 dropout=None,
                 graph_converter=None,
                 target_scaler=DummyScaler(),
                 optimizer_kwargs=None,
                 dropout_on_predict=False
                 ):

        # Build the MEG Model
        model = make_megnet_model(nfeat_edge=nfeat_edge,
                                  nfeat_global=nfeat_global,
                                  nfeat_node=nfeat_node,
                                  nblocks=nblocks,
                                  n1=n1,
                                  n2=n2,
                                  n3=n3,
                                  nvocal=nvocal,
                                  embedding_dim=embedding_dim,
                                  nbvocal=nbvocal,
                                  bond_embedding_dim=bond_embedding_dim,
                                  ngvocal=ngvocal,
                                  global_embedding_dim=global_embedding_dim,
                                  npass=npass,
                                  ntarget=ntarget,
                                  act=act,
                                  is_classification=is_classification,
                                  l2_coef=l2_coef,
                                  dropout=dropout,
                                  dropout_on_predict=dropout_on_predict)

        # Compile the model with the optimizer
        # loss = 'binary_crossentropy' if is_classification else loss
        # loss = 'binary_crossentropy' if is_classification else loss
        # loss = log_mae+mse
        opt_params = {'lr': lr}
        if optimizer_kwargs is not None:
            opt_params.update(optimizer_kwargs)
        # model.compile(Adam(**opt_params), loss, metrics=metrics)
        model.compile(Adam(**opt_params), log_mae, metrics=[mse])
#         model.compile(opt, log_mae, metrics=[mse])
        if graph_converter is None:
            graph_converter = CrystalGraph(cutoff=4, bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5))

        super().__init__(model=model, target_scaler=target_scaler, graph_converter=graph_converter)


def make_megnet_model(nfeat_edge=None, nfeat_global=None, nfeat_node=None, nblocks=3,
                      n1=64, n2=32, n3=16, nvocal=95, embedding_dim=16, nbvocal=None,
                      bond_embedding_dim=None, ngvocal=None, global_embedding_dim=None,
                      npass=3, ntarget=1, act=softplus2, is_classification=False,
                      l2_coef=None, dropout=None, dropout_on_predict=False):
    """Make a MEGNet Model

    Args:
        nfeat_edge: (int) number of bond features
        nfeat_global: (int) number of state features
        nfeat_node: (int) number of atom features
        nblocks: (int) number of MEGNetLayer blocks
        n1: (int) number of hidden units in layer 1 in MEGNetLayer
        n2: (int) number of hidden units in layer 2 in MEGNetLayer
        n3: (int) number of hidden units in layer 3 in MEGNetLayer
        nvocal: (int) number of total element
        embedding_dim: (int) number of embedding dimension
        nbvocal: (int) number of bond types if bond attributes are types
        bond_embedding_dim: (int) number of bond embedding dimension
        ngvocal: (int) number of global types if global attributes are types
        global_embedding_dim: (int) number of global embedding dimension
        npass: (int) number of recurrent steps in Set2Set layer
        ntarget: (int) number of output targets
        act: (object) activation function
        l2_coef: (float or None) l2 regularization parameter
        is_classification: (bool) whether it is a classification task
        dropout: (float) dropout rate
        dropout_on_predict (bool): Whether to use dropout during prediction and training
    Returns:
        (Model) Keras model, ready to run
    """

    # Get the setting for the training kwarg of Dropout
    dropout_training = True if dropout_on_predict else None

    # Create the input blocks
    int32 = 'int32'
    if nfeat_node is None:
        x1 = Input(shape=(None,), dtype=int32)  # only z as feature
        x1_ = Embedding(nvocal, embedding_dim)(x1)
    else:
        x1 = Input(shape=(None, nfeat_node))
        x1_ = x1
    if nfeat_edge is None:
        x2 = Input(shape=(None,), dtype=int32)
        x2_ = Embedding(nbvocal, bond_embedding_dim)(x2)
    else:
        x2 = Input(shape=(None, nfeat_edge))
        x2_ = x2
    if nfeat_global is None:
        x3 = Input(shape=(None,), dtype=int32)
        x3_ = Embedding(ngvocal, global_embedding_dim)(x3)
    else:
        x3 = Input(shape=(None, nfeat_global))
        x3_ = x3
    x4 = Input(shape=(None,), dtype=int32)
    x5 = Input(shape=(None,), dtype=int32)
    x6 = Input(shape=(None,), dtype=int32)
    x7 = Input(shape=(None,), dtype=int32)
    if l2_coef is not None:
        reg = l2(l2_coef)
    else:
        reg = None

    # two feedforward layers
    def ff(x, n_hiddens=[n1, n2]):
        out = x
        for i in n_hiddens:
            out = Dense(i, activation=act, kernel_regularizer=reg)(out)
        return out

    # a block corresponds to two feedforward layers + one MEGNetLayer layer
    # Note the first block does not contain the feedforward layer since
    # it will be explicitly added before the block
    def one_block(a, b, c, has_ff=True):
        if has_ff:
            x1_ = ff(a)
            x2_ = ff(b)
            x3_ = ff(c)
        else:
            x1_ = a
            x2_ = b
            x3_ = c
        out = MEGNetLayer(
            [n1, n1, n2], [n1, n1, n2], [n1, n1, n2],
            pool_method='mean', activation=act, kernel_regularizer=reg)(
            [x1_, x2_, x3_, x4, x5, x6, x7])

        x1_temp = out[0]
        x2_temp = out[1]
        x3_temp = out[2]
        if dropout:
            x1_temp = Dropout(dropout)(x1_temp, training=dropout_training)
            x2_temp = Dropout(dropout)(x2_temp, training=dropout_training)
            x3_temp = Dropout(dropout)(x3_temp, training=dropout_training)
        return x1_temp, x2_temp, x3_temp

    x1_ = ff(x1_)
    x2_ = ff(x2_)
    x3_ = ff(x3_)
    for i in range(nblocks):
        if i == 0:
            has_ff = False
        else:
            has_ff = True
        x1_1 = x1_
        x2_1 = x2_
        x3_1 = x3_
        x1_1, x2_1, x3_1 = one_block(x1_1, x2_1, x3_1, has_ff)
        # skip connection
        x1_ = Add()([x1_, x1_1])
        x2_ = Add()([x2_, x2_1])
        x3_ = Add()([x3_, x3_1])
    # set2set for both the atom and bond
    node_vec = Set2Set(T=npass, n_hidden=n3, kernel_regularizer=reg)([x1_, x6])
    edge_vec = Set2Set(T=npass, n_hidden=n3, kernel_regularizer=reg)([x2_, x7])
    # concatenate atom, bond, and global
    final_vec = Concatenate(axis=-1)([node_vec, edge_vec, x3_])
    # final_vec = edge_vec
    print(final_vec.shape)
    if dropout:
        final_vec = Dropout(dropout)(final_vec, training=dropout_training)
    # final dense layers
    final_vec = Dense(n2, activation=act, kernel_regularizer=reg)(final_vec)
    final_vec = Dense(n3, activation=act, kernel_regularizer=reg)(final_vec)
    if is_classification:
        final_act = 'sigmoid'
    else:
        final_act = None
    out = Dense(820, activation=final_act)(final_vec)
    # out = final_vec
    model = Model(inputs=[x1, x2, x3, x4, x5, x6, x7], outputs=out)

    return model


def generate_arrays_from_file(model,batch_size=2):
    graph_path = ['../input/all-graph-12/input_data0','../input/all-graph-12/input-data1','../input/all-graph-2/input-data2','../input/graph-data-3/input-data3']
#     target_path = ['../input/shiyan-graph/train_targets','../input/shiyan-graph-1/train_targets','../input/shiyangraph2/train_targets','../input/shiyangraph3/train_targets']
    while 1:

        for graph_p in graph_path:
            
            with open(graph_p, 'rb') as fp:
                input_data = pickle.load(fp)
                
            cnt = 0
            feature_list_temp=[]
            connection_list_tempp = []
            global_list_temp =[]
            index1_temp= []
            index2_temp = []
            temp_target = []
            X=[]
            for atom_features,bond_features,state_features,index1_list,index2_list,target in zip(*input_data):
                # create Numpy arrays of input dat
                # and labels, from each line in the file
                feature_list_temp.append(atom_features)
                connection_list_tempp.append(bond_features)
                global_list_temp.append(state_features)
                index1_temp.append(index1_list)
                index2_temp.append(index2_list)
                temp_target.append(target)
                
                
                gnode = [0] * len(atom_features)
                gbond = [0] * len(index1_list)

       
                
                
                cnt += 1

                if cnt==batch_size:
                    X.append(feature_list_temp)
                    X.append(connection_list_tempp)
                    X.append(global_list_temp)
                    X.append(index1_temp)
                    X.append(index2_temp)
                  
                    yield ([np.array(feature_list_temp),np.array(connection_list_tempp),np.array(global_list_temp),np.array(index1_temp),np.array(index2_temp),np.array(gnode),np.array(gbond)],np.array(temp_target))
                    cnt = 0
                    X=[]
                    feature_list_temp=[] 
                    connection_list_tempp = []
                    global_list_temp =[]

                    index1_temp= []
                    index2_temp = []
                    temp_target = []
                    
# from megnet.data.molecule import MolecularGraph


pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
# only reading 10% of data for debug
# train = pd.read_csv('../../../../resources/train.csv')
# test = pd.read_csv('../../../../resources/test.csv')

##
## Build molecules from files.xyz
##

obConversion = ob.OBConversion()
#def read_ob_molecule(molecule_name, datadir="../input/champs-scalar-coupling/structures"):
def read_ob_molecule(molecule_name, filename=None):
    mol = ob.OBMol()
    path = f"../input/champs-scalar-coupling/structures/{molecule_name}.xyz"
    if not obConversion.ReadFile(mol, path):
        raise FileNotFoundError(f"Could not read molecule {filename}")
    return mol

# print(read_ob_molecule('dsgdb9nsd_000001'))
# print(train.columns)
# print(train[['molecule_name', 'atom_index_0', 'atom_index_1', 'type',
#        'scalar_coupling_constant']].head(10))
n_cpu = 4
mols={}


    # molecule_names = np.concatenate([train.molecule_name.unique(), test.molecule_name.unique()])
# molecule_names = train.molecule_name.unique()
#     # xyzfiles = [Path(f'../../../../resources/structures/{f}.xyz') for f in molecule_names]
# n = len(molecule_names)
# i=0
# with tqdm(total=n) as pbar:
#     for name in molecule_names:
#         # mol = read_ob_molecule(name)
#             # mol1 = pybel.readfile('xyz', filename).
#             # print(pybel.Molecule(mol))
# #             print(train[train['molecule_name']==res[0]])
# #             print(train[train['molecule_name'] == name][['molecule_name', 'atom_index_0', 'atom_index_1', 'type',
# #                                                          'scalar_coupling_constant']])
# #         mols[name] =(pybel.Molecule(mol), train[train['molecule_name'] == name])
# #         outMDL = obConversion.WriteString(mol)
#         mols[name] = (name, train[train['molecule_name'] == name])
#         if(i==1):
#              break
#         i=i+1
#         pbar.update()
#
#
# # print(list(mols.values())[0])
# mols = list(mols.values())
# import pickle
#
# with open('train_outfile', 'wb') as fp:
#     pickle.dump(mols, fp)

# with open ('../input/new-train-data-set/train_outfile', 'rb') as fp:
#     amols = pickle.load(fp)
# mols=amols[:20]
# print(mols[:2])
# test_mols = amols[25:30]

# with open ('train_graphs', 'rb') as fp:
#     mols = pickle.load(fp)

# for i in mols:
#     print(i)
# jsObj = json.dumps(mols)
#
# fileObject = open('jsonFile.json', 'w')
# fileObject.write(jsObj)
# fileObject.close()



# targets = [[0,0,0,-2.8],[0,0,0,2.4]]
def mse(orig, preds):
    # Mask values for which no scalar coupling exists
    # orig=orig[:114]
    # preds=preds[:114]
    mask = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums = tf.boolean_mask(orig, mask)
    preds = tf.boolean_mask(preds, mask)

    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(nums, preds)))

    return reconstruction_error

def log_mae(orig, preds):
    # orig = orig[114:]
    # preds= preds[114:]
    # Mask values for which no scalar coupling exists
    mask = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums = tf.boolean_mask(orig, mask)
    preds = tf.boolean_mask(preds, mask)

    reconstruction_error = tf.math.log(tf.reduce_mean(tf.abs(tf.subtract(nums, preds))))

    return reconstruction_error



# Define some callbacks, the initial learning rate and the optimizer
# learning_rate = 0.001
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.1
    epochs_drop = 20.0
    lrate = initial_lrate * np.power(drop,
           np.floor((epoch)/epochs_drop))
    tf.print("Learning rate: ", lrate)
    return lrate

# from keras.callbacks import LearningRateScheduler
# lrate = LearningRateScheduler(step_decay)


import keras.backend as K
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
def scheduler(epoch):
    # 每隔100个epoch，学习率减小为原来的1/10
    if epoch % 100 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)
  
# lrate = LearningRateScheduler(step_decay)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, restore_best_weights=True)

lrate  =  ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=10, min_lr=0.000001, verbose = 1)

# opt = tf.optimizers.Adam(learning_rate=learning_rate)


model = MEGNetModel(35, 2, 27, nblocks=1, lr=1e-2,
                    n1=4, n2=32, n3=16, npass=1, ntarget=1,graph_converter=MolecularGraph())
# model = MEGNetModel.from_file("./train_model_1")
# mpnn.compile(opt, log_mae, metrics = [mae, log_mse])



# model.summary()
#
# model.load_weights('../input/shiyan-data-graph/val_mae_00009_0.119766.hdf5')

model.load_weights('../input/tune-model/weights.best.hdf5')
model.train_coupling(mols,epochs=200, verbose=1,batch_size=1)
# model.save("train_model_1")

# ###test data begin#######
#
# test_mols={}
# #     molecule_names = np.concatenate([train.molecule_name.unique(), test.molecule_name.unique()])
# molecule_names = test.molecule_name.unique()
#     # xyzfiles = [Path(f'../../../../resources/structures/{f}.xyz') for f in molecule_names]
# n = len(molecule_names)
# i=0
# with tqdm(total=n) as pbar:
#     for name in molecule_names:
#         # mol = read_ob_molecule(name)
#             # mol1 = pybel.readfile('xyz', filename).
#             # print(pybel.Molecule(mol))
# #             print(train[train['molecule_name']==res[0]])
#
#         test_mols[name] =(name, test[test['molecule_name'] == name])
#         if(i==10):
#             break
#         i=i+1
#         pbar.update()
#
#
# # print(list(mols.values())[0])
# test_mols = list(test_mols.values())
#
# test_graphs,  test_datas= model.get_all_graphs_targets_coupling(test_mols,target_convert=False)
#


# with open('../input/test-dat/test_graphs', 'rb') as fp:
#     test_graphs = pickle.load(fp)

# with open('../input/test-data/test_datas', 'rb') as fp:
#     test_datas = pickle.load(fp)
# for graph,test_data in zip(test_graphs,test_datas):
#     # print(graph)
#     # print(test_data)
#     tt = model.predict_graph(graph)
#     length = len(test_data['index1'])
#     couple_constant_data = tt[:length]
#     print(couple_constant_data)
#     couple_constant_data= couple_constant_data*(max_value-min_value)+min_value
#     print(couple_constant_data)
#     df1 = pd.DataFrame({'index1':test_data['index1'],
#                     'index2':test_data['index2'],
#                     'couple_constant_data':couple_constant_data
#                     })
#     df2=test_data['coupling_attributes']
#     df2.rename(columns={'atom_index_0':'index1', 'atom_index_1':'index2'}, inplace = True)
#     print(df1)
#     print(df2)
#     data=[]
#     for index,row in df2.iterrows():

#         values1 = df1[(df1['index1'] == row['index1']) &(df1['index2'] == row['index2'])]['couple_constant_data'].tolist()[0]
#         values2 = df1[(df1['index1'] == row['index2']) & (df1['index2'] == row['index1'])]['couple_constant_data'].tolist()[0]

#         # coupling_arr0 = coupling_attr[(coupling_attr['atom_index_0'] == bid) & (coupling_attr['atom_index_1'] == eid)]
#         # one = values1['couple_constant_data'].tolist()[0]
#         # two = values2['couple_constant_data'].tolist()[0]

#         data.append((values1+values2)/2)

#     print("################")
#     df2['couple_data_value'] = data

#     print(df2)
#     # df= pd.merge(df2,df1)

#     # print(df)
#     df2.to_csv('test_result.csv',index = False,header=0,mode='a')
# #     print('fininsed####################')
# # #     # print(df[['molecule_name','index1', 'index2', 'couple_constant_data',
# #     #    'type', 'scalar_coupling_constant']])
# print('fininsed####################')

# predicted_atom = [model.predict_structure(x) for x in structures]
# print(predicted_atom)




In [ ]:
import keras
print(tensorflow.__version__)

In [ ]:
# import pickle

# # with open('../input/shiyan-graph/train_graphs', 'rb') as fp:
# #     data_graph = pickle.load(fp)

# # with open('../input/shiyan-graph/train_targets', 'rb') as fp:
# #     data_target = pickle.load(fp)

# with open('../input/shiyan-graph-1/train_graphs', 'rb') as fp:
#     data_graphs = pickle.load(fp)

# with open('../input/shiyan-graph-1/train_targets', 'rb') as fp:
#     data_targets = pickle.load(fp)
    
# # data_graph=None
# # data_targets=None
# # #         print(len(data_targets))
# print(data_targets[0])
# for target in  data_targets:
#     target =  (target-min_value)/(max_value -min_value) 
# data_targets[:2]

In [ ]:
!pip show keras

In [ ]:
!ls ../input/